In [14]:
import numpy as np
from skimage.io import imread
from skimage.filters import threshold_otsu
from skimage.transform import resize
#import cPickle
from matplotlib import pyplot as plt
from skimage.morphology import closing, square
from skimage.measure import regionprops
from skimage import restoration
from skimage import measure
from skimage.color import label2rgb
import matplotlib.patches as mpatches
import os
import skimage
from skimage import io
from skimage.viewer import ImageViewer

In [21]:
filename = os.path.join(skimage.data_dir, 'P:\\School\\Python_Fall2015\\Project\\sampleImage.jpg')
chk = io.imread(filename,as_grey = True)

In [66]:
#Process one
image = restoration.denoise_tv_chambolle(chk, weight=0.1)
thresh = threshold_otsu(image)
bw = closing(image > thresh, square(2))
cleared = bw.copy()
viewer = ImageViewer(cleared)
viewer.show()

[]

In [40]:
#Process two
label_image = measure.label(cleared)
borders = np.logical_xor(bw, cleared)
label_image[borders] = -1


coordinates = []
i=0
        
for region in regionprops(label_image):
    if region.area > 10:
        minr, minc, maxr, maxc = region.bbox
        margin = 3
        minr, minc, maxr, maxc = minr-margin, minc-margin, maxr+margin, maxc+margin
        roi = chk[minr:maxr, minc:maxc]
        if roi.shape[0]*roi.shape[1] == 0:
            continue
        else:
            if i==0:
                samples = resize(roi, (20,20))
                coordinates.append(region.bbox)
                i+=1
            elif i==1:
                roismall = resize(roi, (20,20))
                samples = np.concatenate((samples[None,:,:], roismall[None,:,:]), axis=0)
                coordinates.append(region.bbox)
                i+=1
            else:
                roismall = resize(roi, (20,20))
                samples = np.concatenate((samples[:,:,:], roismall[None,:,:]), axis=0)
                coordinates.append(region.bbox)

candidates = {
                    'fullscale': samples,          
                    'flattened': samples.reshape((samples.shape[0], -1)),
                    'coordinates': np.array(coordinates)
                    }


In [73]:
from scipy.misc import toimage
print(candidates['coordinates'])
test = chk[59:113,628:695]
toimage(test).show()

[[ 59 628 113 695]
 [ 89 654  97 660]
 [ 90 613 108 634]
 ..., 
 [743 579 756 583]
 [748 525 752 534]
 [750 564 757 567]]


In [62]:
from skimage.segmentation import clear_border
from skimage import data
image = data.coins()[50:-50, 50:-50]

# apply threshold
thresh = threshold_otsu(image)
bw = closing(image > thresh, square(3))

# remove artifacts connected to image border
cleared = bw.copy()
clear_border(cleared)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ..., 
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], dtype=bool)

In [74]:
print(samples)

[[[ 0.91980824  0.91480824  0.91941608 ...,  0.90787373  0.91217608
    0.90345059]
  [ 0.82120353  0.94394863  0.91590941 ...,  0.90003059  0.90443098
    0.90090157]
  [ 0.70494039  0.74141098  0.89797961 ...,  0.91032471  0.91609765
    0.9047251 ]
  ..., 
  [ 0.88570902  0.79092766  0.59450892 ...,  0.56806431  0.63655311
    0.52884924]
  [ 0.86716925  0.84163934  0.86165686 ...,  0.61839608  0.78527621
    0.83637575]
  [ 0.91621751  0.90628588  0.90090971 ...,  0.75692824  0.8580102
    0.84702196]]

 [[ 0.13289647  0.14068253  0.15654233 ...,  0.30839033  0.32239033
    0.26537893]
  [ 0.13289647  0.1680946   0.1809259  ...,  0.3845281   0.39249478
    0.31824472]
  [ 0.13289647  0.17872455  0.21153637 ...,  0.39543294  0.39610365
    0.31724063]
  ..., 
  [ 0.39981004  0.50009831  0.50060196 ...,  0.71269373  0.73395949
    0.59850933]
  [ 0.41799241  0.50947103  0.4899418  ...,  0.7006192   0.70624231
    0.56799285]
  [ 0.3571832   0.4282906   0.401008   ...,  0.53437933  0.